## 7. Concurrency and Parallelism

### 54 Use `Lock` to Prevent Data Races in Threads

In [1]:
class Counter:
    def __init__(self):
        self.count = 0

    def increment(self, offset):
        self.count += offset

In [2]:
def worker(sensor_index, how_many, counter):
    # I have a barrier in here so the workers synchronize
    # when they start counting, otherwise it's hard to get a race
    # because the overhead of starting a thread is high.
    BARRIER.wait()
    for _ in range(how_many):
        # Read from the sensor
        # Nothing actually happens here, but this is where
        # the blocking I/O would go.
        counter.increment(1)

In [3]:
from threading import Barrier
from threading import Thread

In [4]:
how_many = 10**5

In [5]:
BARRIER = Barrier(5)

counter = Counter()

threads = []
for i in range(5):
    thread = Thread(target=worker,
                    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

expected = how_many * 5
found = counter.count
print(f'Counter should be {expected}, got {found}')

Counter should be 500000, got 311001


In [6]:
counter.count += 1

# is equivalent to

value = getattr(counter, 'count')
result = value + 1
setattr(counter, 'count', result)

In [7]:
# Running in Thread A
value_a = getattr(counter, 'count')
# Context switch to Thread B
value_b = getattr(counter, 'count')
result_b = value_b + 1
setattr(counter, 'count', result_b)
# Context switch back to Thread A
result_a = value_a + 1
setattr(counter, 'count', result_a)

In [8]:
from threading import Lock

class LockingCounter:
    def __init__(self):
        self.lock = Lock()
        self.count = 0

    def increment(self, offset):
        with self.lock:
            self.count += offset

In [9]:
BARRIER = Barrier(5)

counter = LockingCounter()

for i in range(5):
    thread = Thread(target=worker,
                    args=(i, how_many, counter))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

expected = how_many * 5
found = counter.count
print(f'Counter should be {expected}, got {found}')

Counter should be 500000, got 500000


> - 파이썬에는 GIL이 있지만, 파이썬 프로그램 코드는 여전히 여러 스레드 사이에 일어나는 데이터 경합으로부터 자신을 보호해야 한다.
> - 코드에서 여러 스레드가 상호 배제 락(뮤텍스) 없이 같은 객체를 변경하도록 허용하면 코드가 데이터 구조를 오염시킬 것이다.
> - 여러 스레드 사이에서 프로그램의 불변 조건을 유지하려면 `threading` 내장 모듈의 `Lock` 클래스를 활용하라.